In [ ]:
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

STEP 1: Install Required Libraries  
PURPOSE:
Installing libraries needed for embedding generation (Sentence Transformers), vector

In [4]:
!pip install -q sentence-transformers faiss-cpu requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00


STEP 2: Set Your Groq API Key

In [6]:
GROQ_API_KEY = "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"


STEP 3: Define Sample Code Chunks

In [5]:
code_chunks = [
    "def sort_list(lst): return sorted(lst)",
    "def reverse_string(s): return s[::-1]",
    "def factorial(n): return 1 if n == 0 else n * factorial(n - 1)",
    "def is_prime(n): return all(n % i != 0 for i in range(2, int(n ** 0.5) + 1)) and n > 1"
]


STEP 4: Embed Code Snippets and Index Using FAISS

In [7]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embed_model = SentenceTransformer('all-MiniLM-L6-v2')
chunk_embeddings = embed_model.encode(code_chunks, convert_to_numpy=True)

index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(chunk_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

STEP 5: Define Groq LLM Call Function

In [8]:
import requests

def call_groq_llm(prompt, api_key=GROQ_API_KEY):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3-8b-8192",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.3
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.status_code} - {response.text}"


STEP 6: Code QnA Bot Function

In [9]:
def ask_code_bot(user_query, top_k=2):
    # Embed and retrieve relevant code
    query_embedding = embed_model.encode([user_query])
    D, I = index.search(np.array(query_embedding), k=top_k)
    retrieved_snippets = [code_chunks[i] for i in I[0]]

    # Build prompt
    prompt = f"""You're a helpful coding assistant.
User asked: {user_query}
Relevant code snippets:
{chr(10).join(retrieved_snippets)}
Explain or suggest improvements.
"""

    # Get answer from Groq LLM
    return call_groq_llm(prompt)

# Example:
response = ask_code_bot("How can I check if a number is a palindrome?")
print(response)


To check if a number is a palindrome, you can convert the number to a string, reverse the string, and then compare it to the original string. If they are the same, then the number is a palindrome.

Here's a possible implementation:
```
def is_palindrome(n):
    s = str(n)
    reversed_s = reverse_string(s)
    return s == reversed_s
```
This implementation uses the `reverse_string` function you provided, which is a nice touch!

However, there's a small issue: the `factorial` function is not relevant to this problem, so you might want to remove it or rename it to something like `reverse_string` to avoid confusion.

Also, if you want to make the code more efficient, you can use slicing to reverse the string instead of calling the `reverse_string` function:
```
def is_palindrome(n):
    s = str(n)
    return s == s[::-1]
```
This code is equivalent to the previous one, but it's a bit more concise and efficient.

Finally, if you want to make the code more readable, you could consider addin

STEP 7: Evaluation on 20 Queries

In [11]:
import time
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

# --------------------------
# 👇 Replace these with your real data/models
code_chunks = [
    "def is_palindrome(n): return str(n) == str(n)[::-1]",
    "def factorial(n): return 1 if n == 0 else n * factorial(n-1)",
    "def fibonacci(n): a, b = 0, 1\nfor _ in range(n): a, b = b, a+b\nreturn a",
    "def reverse_string(s): return s[::-1]",
    "def gcd(a, b): return a if b == 0 else gcd(b, a % b)"
]
embed_model = SentenceTransformer("all-MiniLM-L6-v2")  # or your embed model
index = faiss.IndexFlatL2(384)
index.add(np.array(embed_model.encode(code_chunks)))

# --------------------------
# Your core Groq LLM call function
def call_groq_llm(prompt, api_key=GROQ_API_KEY):
    import requests
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3-8b-8192",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.status_code}"

# QnA bot function
def ask_code_bot(user_query, top_k=2):
    query_embedding = embed_model.encode([user_query])
    D, I = index.search(np.array(query_embedding), k=top_k)
    retrieved_snippets = [code_chunks[i] for i in I[0]]
    prompt = f"""You're a helpful coding assistant.
User asked: {user_query}
Relevant code snippets:
{chr(10).join(retrieved_snippets)}
Explain or suggest improvements.
"""
    return call_groq_llm(prompt), I[0]  # Return both answer and indices

# --------------------------
# Test set of 20 queries with known expected snippet index (manual labels)
test_data = [
    {"query": "How to check if number is palindrome?", "expected": 0},
    {"query": "Factorial code?", "expected": 1},
    {"query": "Give fibonacci sequence code", "expected": 2},
    {"query": "How to reverse a string in Python?", "expected": 3},
    {"query": "Python function for GCD?", "expected": 4},
    {"query": "Check if word is same backwards", "expected": 0},
    {"query": "Recursive factorial in Python", "expected": 1},
    {"query": "Find nth Fibonacci number", "expected": 2},
    {"query": "Reverse the string", "expected": 3},
    {"query": "Greatest common divisor code", "expected": 4},
    {"query": "Is 121 a palindrome?", "expected": 0},
    {"query": "Code to compute factorial", "expected": 1},
    {"query": "Fib series in loop", "expected": 2},
    {"query": "Reverse word order in string", "expected": 3},
    {"query": "Find gcd of two numbers", "expected": 4},
    {"query": "Palindrome checker function", "expected": 0},
    {"query": "Calculate 5!", "expected": 1},
    {"query": "Fibonacci number at index 5", "expected": 2},
    {"query": "How to flip a string?", "expected": 3},
    {"query": "How to find HCF in Python?", "expected": 4}
]

# --------------------------
# Run the test
y_true = []
y_pred = []
helpfulness_scores = []
latencies = []

for item in test_data:
    query = item["query"]
    expected = item["expected"]

    start = time.time()
    response, indices = ask_code_bot(query)
    end = time.time()

    y_true.append(expected)
    y_pred.append(indices[0])
    helpfulness_scores.append(0.9)  # ⬅️ Manual rating or random for now
    latencies.append(end - start)

# --------------------------
# Metrics Calculation
correct_top1 = sum([1 for yt, yp in zip(y_true, y_pred) if yt == yp])
top_k_accuracy = correct_top1 / len(y_true)
avg_helpfulness = sum(helpfulness_scores) / len(helpfulness_scores)
avg_latency = sum(latencies) / len(latencies)
f1 = f1_score(y_true, y_pred, average='macro')

# --------------------------
# Results
print(f"✅ Top-2 Retrieval Accuracy: {top_k_accuracy:.2f}")
print(f"🌟 Average Helpfulness Score: {avg_helpfulness:.2f}")
print(f"⚡ Average Latency: {avg_latency:.4f} seconds")
print(f"🎯 F1-Score: {f1:.2f}")


✅ Top-2 Retrieval Accuracy: 0.95
🌟 Average Helpfulness Score: 0.90
⚡ Average Latency: 0.6963 seconds
🎯 F1-Score: 0.95
